In [1]:
!pip install BeautifulSoup4

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [2]:
!pip install geocoder

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [3]:
import pandas as pd
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup

# Part 1 - Extracting data and creating the dataframe

In [4]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = urllib.request.Request(url, headers={ 'User-Agent': 'Mozilla/5.0' })
req = urllib.request.urlopen(req)
tree = BeautifulSoup(req, 'html5lib')

In [5]:
table = tree.find_all('tr')

In [6]:
codes = []
borough = []
neigh = []

In [7]:
for x in table:
    for y in x.find_all('td'):
        text =y.get_text().strip()
        if 'Not assigned' in text:
            pass
        else:
            words = text[3:]
            texts = (words.strip(')').split('('))
            try:
                text1 = texts[0]
                text2 = texts[1]
                pcode = text[:3]
                if text2 == 'Not assigned':
                    text2 = text1
                    codes.append(pcode)
                    borough.append(text1)
                    neigh.append(text2)
                else:
                    codes.append(pcode)
                    borough.append(text1)
                    neigh.append(text2)
            except:
                pass

In [8]:
table = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood'])

In [9]:
table.head()

,PostalCode,Borough,Neighborhood


In [10]:
table['PostalCode'] = codes
table['Borough'] = borough
table['Neighborhood'] = neigh

In [11]:
table.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Queen's Park,Ontario Provincial Government


# PART 2 - Adding latitude and longitude

In [12]:
geo = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')

In [13]:
lat1 = []
long1 =[]
codes2 = []
bor=[]
neighb=[]


for x,y,z in zip(geo['Postal Code'],geo['Latitude'],geo['Longitude']):
    for n,m,o in zip(table['PostalCode'],table['Borough'],table['Neighborhood']):
        if x == n:
            lat = y
            long = z
            lat1.append(lat)
            long1.append(long)
            codes2.append(n)
            bor.append(m)
            neighb.append(o.replace(')',' '))
        else:
            pass
        

In [14]:
table2 = pd.DataFrame()
table2['Code'] = codes2
table2['Borough'] = bor
table2['Neighborhood'] = neighb
table2['Latitude'] = lat1
table2['Longitude'] = long1

In [15]:
table2.head(10)

,Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park,43.727929,-79.262029
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge,43.711112,-79.284577
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West,43.716316,-79.239476
9,M1N,Scarborough,Birch Cliff / Cliffside West,43.692657,-79.264848


# Part 3 - Clustering

In [17]:
!pip install folium

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 94 kB 6.2 MB/s  eta 0:00:01


In [18]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [19]:
address = 'Toronto City, TOR'

geolocator = Nominatim(user_agent="ny_explorer")
location1 = geolocator.geocode(address)
latitude1 = location1.latitude
longitude1 = location1.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude1, longitude1))

The geograpical coordinate of Toronto City are 43.7370584, -79.2442535.


In [20]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude1, longitude1], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(table2['Latitude'], table2['Longitude'], table2['Borough'], table2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [21]:
df2 = table2.drop(['Code','Borough','Neighborhood'],axis = 1)

In [24]:
df2.head()

,Latitude,Longitude
0,43.806686,-79.194353
1,43.784535,-79.160497
2,43.763573,-79.188711
3,43.770992,-79.216917
4,43.773136,-79.239476


In [25]:
df2.values[:]

array([[ 43.8066863, -79.1943534],
       [ 43.7845351, -79.1604971],
       [ 43.7635726, -79.1887115],
       [ 43.7709921, -79.2169174],
       [ 43.773136 , -79.2394761],
       [ 43.7447342, -79.2394761],
       [ 43.7279292, -79.2620294],
       [ 43.7111117, -79.2845772],
       [ 43.716316 , -79.2394761],
       [ 43.692657 , -79.2648481],
       [ 43.7574096, -79.273304 ],
       [ 43.7500715, -79.2958491],
       [ 43.7942003, -79.2620294],
       [ 43.7816375, -79.3043021],
       [ 43.8152522, -79.2845772],
       [ 43.7995252, -79.3183887],
       [ 43.8361247, -79.2056361],
       [ 43.8037622, -79.3634517],
       [ 43.7785175, -79.3465557],
       [ 43.7869473, -79.385975 ],
       [ 43.7574902, -79.3747141],
       [ 43.789053 , -79.4084928],
       [ 43.7701199, -79.4084928],
       [ 43.7527583, -79.4000493],
       [ 43.7827364, -79.4422593],
       [ 43.7532586, -79.3296565],
       [ 43.7459058, -79.352188 ],
       [ 43.7258997, -79.340923 ],
       [ 43.7543283,

In [28]:
import numpy as np

In [29]:
from sklearn.preprocessing import StandardScaler

X = df2.values[:]
X = np.nan_to_num(X)
cluster_dataset = StandardScaler().fit_transform(X)
cluster_dataset

array([[ 1.95523876,  2.09777597],
       [ 1.53094905,  2.44798852],
       [ 1.12942801,  2.15613628],
       [ 1.271543  ,  1.86437197],
       [ 1.3126078 ,  1.6310228 ],
       [ 0.76859251,  1.6310228 ],
       [ 0.44670528,  1.39772948],
       [ 0.12457862,  1.16449306],
       [ 0.2242631 ,  1.6310228 ],
       [-0.22890739,  1.3685726 ],
       [ 1.01138034,  1.28110403],
       [ 0.87082451,  1.04789553],
       [ 1.71607875,  1.39772948],
       [ 1.47544769,  0.96045696],
       [ 2.11931218,  1.16449306],
       [ 1.81807323,  0.81474393],
       [ 2.51910935,  1.98106673],
       [ 1.89922981,  0.3486083 ],
       [ 1.41568642,  0.523382  ],
       [ 1.57715295,  0.1156253 ],
       [ 1.01292418,  0.23210904],
       [ 1.61748606, -0.1173008 ],
       [ 1.25483666, -0.1173008 ],
       [ 0.92228816, -0.0299605 ],
       [ 1.4964963 , -0.46658445],
       [ 0.93187104,  0.69818881],
       [ 0.79103364,  0.465121  ],
       [ 0.40783172,  0.58164715],
       [ 0.95236034,

In [31]:
num_clusters = 6

k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init=12)
k_means.fit(cluster_dataset)
labels = k_means.labels_

print(labels)

[3 3 3 3 3 3 2 2 2 2 3 2 3 3 3 3 3 0 0 0 0 0 0 0 0 2 2 2 0 0 1 1 1 1 2 2 2
 2 2 2 2 4 4 4 2 2 2 2 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 2 4 4 4 4 4 4 4 1 1 4
 1 4 4 4 4 1 1 5 5 5 5 4 5 4 5 5 5 5 5 5 5 5 1 1 1 1 1 1 1]


In [33]:
df2["Labels"] = labels
df2.head(5)

,Latitude,Longitude,Labels
0,43.806686,-79.194353,3
1,43.784535,-79.160497,3
2,43.763573,-79.188711,3
3,43.770992,-79.216917,3
4,43.773136,-79.239476,3


In [36]:
table2['Labels'] = labels

In [37]:
table2.head()

,Code,Borough,Neighborhood,Latitude,Longitude,Labels
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353,3
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,3
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711,3
3,M1G,Scarborough,Woburn,43.770992,-79.216917,3
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,3


# I tried to see if clustering the data would result in them matching their actual boroughs

In [42]:
# create map
map_clusters = folium.Map(location=[latitude1, longitude1], zoom_start=11)

# set color scheme for the clusters
x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in range(num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, borough, poi, cluster in zip(table2['Latitude'], table2['Longitude'], table2['Borough'],table2['Neighborhood'], table2['Labels']):
    label = folium.Popup(str(borough)+ ' ' +str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Turns out it they don't classify boroughs according to their proximity.